In [1]:
from dataclasses import dataclass
from typing import Dict, List, Optional
import copy
import re
from tqdm.auto import tqdm
from abc import ABC, abstractmethod
import math

import polars as pl
import numpy as np
from numba import njit, prange
from scipy.stats import spearmanr, rankdata

from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.base import clone

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# from torch.profiler import profile, ProfilerActivity, record_function

from CONFIG import CONFIG
from PREPROCESSOR_V2 import PREPROCESSOR
from FEATURE_ENGINEERING_V2 import FEATURE_ENGINEERING
from SEQUENTIAL_NN_MODEL import CNNTransformerModel, GRUModel, LSTMModel, PureTransformerModel
from CROSS_SECTIONAL_NN_MODEL import DeepMLPModel, LinearModel, ResidualMLPModel
from LOSS import CombinedICIRLoss

import time


def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result

    return wrapper

In [2]:
# # --- Prepare DataLoader ---
# # Create the dataset

# train_x = pl.scan_csv(CONFIG.TRAIN_X_PATH).filter(pl.col("date_id") <= CONFIG.MAX_TRAIN_DATE)
# train_y = pl.scan_csv(CONFIG.TRAIN_Y_PATH).filter(pl.col("date_id") <= CONFIG.MAX_TRAIN_DATE).fill_null(0).collect()

# train_x = PREPROCESSOR(df=train_x)
# train_x.clean()
# train_x = train_x.transform().lazy()

# train_x = FEATURE_ENGINEERING(df=train_x)
# train_x = train_x.create_all_features().collect().pivot(index=CONFIG.DATE_COL, on=["type", "instr"])
# train_x = train_x.rename({col: re.sub(r'[{",}]', "", col).replace(" ", "_").replace(",", "_") for col in train_x.columns})
# train_x = train_x.select(set(CONFIG.IMPT_COLS + [CONFIG.DATE_COL]))

In [3]:
def rank_correlation_sharpe(targets, predictions) -> float:
    """
    Calculates the rank correlation between predictions and target values,
    and returns its Sharpe ratio (mean / standard deviation).

    :param merged_df: DataFrame containing prediction columns (starting with 'prediction_')
                      and target columns (starting with 'target_')
    :return: Sharpe ratio of the rank correlation
    :raises ZeroDivisionError: If the standard deviation is zero
    """
    correlations = []

    for i, (pred_row, target_row) in enumerate(zip(predictions, targets)):
        # Find valid (non-NaN) assets for this timestep
        valid_mask = ~np.isnan(target_row)
        valid_pred = pred_row[valid_mask]
        valid_target = target_row[valid_mask]

        if np.std(pred_row) == 0 or np.std(target_row) == 0:
            raise ZeroDivisionError("Zero standard deviation in a row.")

        rho = np.corrcoef(rankdata(valid_pred, method="average"), rankdata(valid_target, method="average"))[0, 1]
        correlations.append(rho)

    daily_rank_corrs = np.array(correlations)
    std_dev = daily_rank_corrs.std(ddof=0)
    if std_dev == 0:
        raise ZeroDivisionError("Denominator is zero, unable to compute Sharpe ratio.")

    sharpe_ratio = daily_rank_corrs.mean() / std_dev
    return float(sharpe_ratio)

In [4]:
class BaseFinancialDataset(Dataset, ABC):
    """Base class for financial datasets"""

    def __init__(self, X: pl.DataFrame, y: pl.DataFrame, date_column: str = CONFIG.DATE_COL):
        """
        Base initialization

        Args:
            data: Preprocessed DataFrame (scaling already done)
            target_columns: List of target column names (424 targets)
            feature_columns: List of feature column names
            date_column: Name of date identifier column
        """
        self.X = X.clone()
        self.y = y.clone()
        self.date_column = date_column

        # Sort by date
        self.X = self.X.sort(by=CONFIG.DATE_COL)
        self.y = self.y.sort(by=CONFIG.DATE_COL)
        self.unique_dates = sorted(self.X[self.date_column].unique())
        self.device = torch.device("cpu")

        self.lag_featues = CONFIG.LAG_FEATURES

        self._prepare_samples()

    def _prepare_samples(self):
        self.samples = {}
        for lag, features in self.lag_featues.items():
            X = self.X.select(features).to_numpy()

            # self.num_features = self.X.shape[-1]
            # Split continuous and categorical features
            continuous_data = torch.tensor(X, dtype=torch.float32)
            continuous_data = torch.nan_to_num(continuous_data, 0)
            self.samples[lag] = continuous_data

        self.dates = torch.tensor(self.unique_dates, dtype=torch.int16)
        self.y = torch.tensor(self.y.drop(CONFIG.DATE_COL).to_numpy(), dtype=torch.float32)

        self.unique_date, self.inverse_indices, self.counts = torch.unique(self.dates, return_inverse=True, return_counts=True)

        self.n_unique_dates = len(self.unique_date)

    @abstractmethod
    def __getitem__(self, idx):
        """Get item - implemented by subclasses"""
        pass

In [5]:
class SequentialDataset(BaseFinancialDataset):
    """Dataset for sequential models (LSTM, Transformers, CNN)"""

    def __init__(self, X: pl.DataFrame, Y: pl.DataFrame, date_column: str = CONFIG.DATE_COL, prediction_horizon: int = 1):
        """
        Sequential dataset for temporal models

        Args:
            data: Preprocessed DataFrame
            target_columns: Target column names
            feature_columns: Feature column names
            date_column: Date identifier column
            sequence_length: Number of time steps in sequence
            prediction_horizon: Steps ahead to predict (usually 1)
        """
        self.prediction_horizon = prediction_horizon

        super().__init__(X, Y)

        self._generate_sequence()

    def _generate_sequence(self):
        self.sequence_x = {1: [], 2: [], 3: [], 4: []}
        self.sequence_y = []

        for date in range(max(CONFIG.LAG_SEQ_LEN.values()), self.n_unique_dates):
            for lag, seq in CONFIG.LAG_SEQ_LEN.items():
                sample = self.samples[lag]
                self.sequence_x[lag].append(sample[date - seq : date])

            self.sequence_y.append(self.y[date - 1])
        for lag, seq in self.sequence_x.items():
            self.sequence_x[lag] = torch.stack(seq)
        self.sequence_y = torch.stack(self.sequence_y)

    def __len__(self):
        return self.n_unique_dates - max(CONFIG.LAG_SEQ_LEN.values())

    def __getitem__(self, idx):
        """Get sequence, target, and date_id"""
        continuous_seq = {lag: self.sequence_x[lag][idx] for lag in self.sequence_x.keys()}  # (seq_len, N_FEATURES)
        target = self.sequence_y[idx]  # (424,)

        return continuous_seq, target

In [6]:
def flatten_collate_fn(batch: list) -> dict[str, torch.Tensor]:
    """
    Collate function for DataLoader to flatten the batch.

    Args:
        batch (list): List of tuples containing tensors.

        tuple[torch.Tensor]: Flattened tensors (type, instr, X, y).
    """
    X, curr_y = zip(*batch)
    lag_keys = X[0].keys()

    # Stack each lag sequence across the batch
    continuous_batch = {
        lag: torch.stack([x_dict[lag] for x_dict in X])  # shape: [B, seq_len, num_features]
        for lag in lag_keys
    }

    curr_y = torch.stack(curr_y)

    return {"continuous": continuous_batch, "current": curr_y}


In [7]:
# seq_val_dataset = SequentialDataset(df_valid, df_valid_current_y)
# seq_val_dataloader = DataLoader(
#     seq_val_dataset,
#     batch_size=6,
#     shuffle=False,
#     collate_fn=flatten_collate_fn,
#     pin_memory=True,
#     # num_workers=6,
#     # persistent_workers=True,
#     # prefetch_factor=2,
#     drop_last=True,
# )

In [8]:
class LagSpecificEnsemble(nn.Module):
    """Ensemble of multiple architectures for each lag"""

    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()

        # seq models
        self.cnn_transformer = CNNTransformerModel(input_dim, hidden_dim, output_dim, CONFIG.SEQ_LEN)
        self.gru_model = GRUModel(input_dim, hidden_dim, output_dim)
        self.lstm_model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers=2)
        self.pure_transformer = PureTransformerModel(input_dim, hidden_dim, output_dim)

        # cross sectional models
        self.mlp = DeepMLPModel(input_dim, [64, 32], output_dim)
        self.linear = LinearModel(input_dim, output_dim)
        self.residual = ResidualMLPModel(input_dim, hidden_dim, output_dim)

        # Ensemble weights (learnable)
        self.ensemble_weights = nn.Parameter(torch.ones(3) / 3)

        self.ensemble_dropout = nn.Dropout(0.3)
        self.prediction_dropout = nn.Dropout(0.3)
        self.input_dropout = nn.Dropout(0.3)

    def forward(
        self,
        x_seq,
    ):
        x_seq = self.input_dropout(x_seq)
        # Get predictions from all models
        x_cs = x_seq[:, -1, :]
        # out1 = self.cnn_transformer(x_seq)
        out2 = self.gru_model(x_seq)
        # out3 = self.lstm_model(x_seq)
        # out4 = self.pure_transformer(x_seq)
        out5 = self.mlp(x_cs)
        # out6 = self.linear(x_cs)
        # out7 = self.residual(x_cs)

        individual_outputs = [out2, out5]  # out1, out2, out3, out4, out5, out6, out7
        individual_outputs = [self.prediction_dropout(out) for out in individual_outputs]

        dropped_weights = self.ensemble_dropout(self.ensemble_weights)
        weights = F.softmax(dropped_weights, dim=0)

        ensemble_output = torch.zeros_like(individual_outputs[0])  # (batch_size, 424)
        for w, out in zip(weights, individual_outputs):
            ensemble_output += w * out

        entropy_reg = self.entropy_regularization(weights)
        return ensemble_output + 0.01 * entropy_reg

    def entropy_regularization(self, weights):
        """Penalize overly confident ensemble weights"""
        return -torch.sum(weights * torch.log(weights + 1e-8))  # Entropy term


class HierarchicalModel(nn.Module):
    """Complete hierarchical model with variable targets per lag"""

    def __init__(self, input_dim: Dict[int, int], lag_target_sizes: Dict[int, int] = CONFIG.LAGS_TARGET):
        super().__init__()

        self.lag_target_sizes = lag_target_sizes
        self.num_lags = len(lag_target_sizes)

        # Lag-specific encoders with different configurations (ensemble models for each lag)
        self.lag_encoders = nn.ModuleList(
            [
                LagSpecificEnsemble(
                    input_dim=input_dim[lag],
                    hidden_dim=64,  # More capacity for longer lags
                    output_dim=lag_target_sizes[lag],
                )
                for lag in lag_target_sizes.keys()
            ]
        )

    def forward(self, inputs: Dict[str, torch.Tensor]):
        """
        inputs: Dict with keys 'lag_1', 'lag_2', etc.
        Each tensor shape: (batch, seq_len, input_dim)
        """
        lag_features = []

        # Process each lag using the ensemble models
        for i, (lag, encoder) in enumerate(zip(self.lag_target_sizes.keys(), self.lag_encoders)):
            lag_key = lag
            if lag_key in inputs:
                features = encoder(inputs[lag_key])
                lag_features.append(features)
            else:
                print("missing")
                # Handle missing lags (e.g., during inference)
                batch_size = list(inputs.values())[0].size(0)
                dummy_features = torch.zeros(batch_size, 64, device=next(encoder.parameters()).device)
                lag_features.append(dummy_features)

        # Return all individual lag predictions directly (no meta combiner)
        return {
            "final_prediction": torch.stack(lag_features).reshape(-1, CONFIG.NUM_TARGET_COLUMNS),  # List of tensors with different sizes for each lag
        }


In [9]:
class NN:
    def __init__(
        self,
        model: HierarchicalModel,
        lr: float = 0.001,
        batch_size: int = 1,
        epochs: int = 100,
        early_stopping_patience: int = 10,
        early_stopping: bool = True,
        lr_patience: int = 2,
        lr_factor: float = 0.5,
        lr_refit: float = 0.001,
        random_seed: int = CONFIG.RANDOM_STATE,
        refit: bool = True,
        **kwargs,
    ) -> None:
        self.lr = lr
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping = early_stopping
        self.lr_patience = lr_patience
        self.lr_factor = lr_factor
        self.lr_refit = lr_refit
        self.random_seed = random_seed
        self.refit = refit

        self.criterion = CombinedICIRLoss()

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        # Different optimizers for different components
        # self.lag_params = list(self.model.lag_encoders.parameters()) + list(self.model.prediction_heads.parameters())
        # self.meta_params = list(self.model.meta_combiner.parameters()) + list(self.model.cross_attention.parameters())

        # self.refit_lag_optimizer = torch.optim.AdamW(self.lag_params, lr=self.lr_refit["lag_optimizer"], weight_decay=0.01)
        # self.refit_meta_optimizer = torch.optim.AdamW(self.meta_params, lr=self.lr_refit["meta_optimizer"], weight_decay=0.01)
        # self.refit_optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr_refit["lag_optimizer"], weight_decay=0.01)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=0.01)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optimizer, T_0=5, T_mult=2, eta_min=1e-6)

        self.refit_optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr_refit, weight_decay=0.01)

        self.best_epoch = None
        self.features = None
        self.kwargs = kwargs

    def fit(self, train_set: tuple, val_set: tuple, retrain_set: tuple, verbose: bool = False) -> None:
        """Fit the model on the training set and validate on the validation set.

        Args:
            train_set (tuple): A tuple containing input data, targets for training.
            val_set (tuple): A tuple containing input data, targets for validation.
            verbose (bool, optional): If True, prints training progress. Defaults to False.
        """
        torch.manual_seed(self.random_seed)

        seq_train_dataset = SequentialDataset(*train_set)
        seq_train_dataloader = DataLoader(
            seq_train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=flatten_collate_fn,
            pin_memory=True,
            # num_workers=2,
            # persistent_workers=True,
            # prefetch_factor=2,
            drop_last=True,
        )

        seq_val_dataset = SequentialDataset(*val_set)
        seq_val_dataloader = DataLoader(
            seq_val_dataset,
            batch_size=1,
            shuffle=False,
            collate_fn=flatten_collate_fn,
            pin_memory=True,
            # num_workers=6,
            # persistent_workers=True,
            # prefetch_factor=2,
            drop_last=True,
        )

        retrain_val_dataset = SequentialDataset(*retrain_set)
        retrain_val_dataloader = DataLoader(
            retrain_val_dataset,
            batch_size=1,
            shuffle=False,
            collate_fn=flatten_collate_fn,
            pin_memory=True,
            # num_workers=6,
            # persistent_workers=True,
            # prefetch_factor=2,
            drop_last=True,
        )

        train_sharpes, val_sharpes = [], []
        if verbose:
            print(f"Device: {self.device}")
            print(
                f"{'Epoch':^5} | {'Train Loss':^10} | {'Train ICIR Loss':^15} | {'Train MSE Loss':^14} | {'Train Ranking Loss':^17} | {'Val Loss':^8} | {'Val ICIR Loss':^13} | {'Val MSE Loss':^12} | {'Val Ranking Loss':^16} | {'Train sharpe':^9} | {'Val sharpe':^7} | {'LR':^7}"
            )
            print("-" * 60)

        min_val_sharpe = -np.inf
        best_epoch = 0
        no_improvement = 0
        best_model = None
        for epoch in range(self.epochs):
            train_loss, train_sharpe, train_icir_loss, train_mse_loss, train_ranking_loss = self.train_one_epoch(seq_train_dataloader, verbose)
            val_loss, val_sharpe, val_icir_loss, val_mse_loss, val_ranking_loss = self.validate_one_epoch(
                seq_val_dataloader, retrain_val_dataloader, verbose
            )

            self.scheduler.step()
            lr_last = self.optimizer.param_groups[0]["lr"]

            # self.lag_scheduler.step()
            # self.meta_scheduler.step()
            # lag_optimizer_lr_last = self.lag_optimizer.param_groups[0]["lr"]
            # meta_optimizer_lr_last = self.meta_optimizer.param_groups[0]["lr"]

            train_sharpes.append(train_sharpe)
            val_sharpes.append(val_sharpe)

            if verbose:
                print(
                    f"{epoch + 1:^5} | {train_loss:^10.4f} | {train_icir_loss:^15.4f} | {train_mse_loss:^14.4f} | {train_ranking_loss:^17.4f} | {val_loss:^8.4f} | {val_icir_loss:^13.4f} | {val_mse_loss:^12.4f} | {val_ranking_loss:^16.4f} | {train_sharpe:^9.4f} | {val_sharpe:^7.4f} | {lr_last:^7.5f}"
                )

            if val_sharpe > min_val_sharpe:
                min_val_sharpe = val_sharpe
                best_model = copy.deepcopy(self.model.state_dict())
                no_improvement = 0
                best_epoch = epoch
            else:
                no_improvement += 1

            if self.early_stopping:
                if no_improvement >= self.early_stopping_patience + 1:
                    self.best_epoch = best_epoch + 1
                    if verbose:
                        print(f"Early stopping on epoch {best_epoch + 1}. Best score: {min_val_sharpe:.4f}")
                    break

        # Load the best model
        if self.early_stopping:
            self.model.load_state_dict(best_model)

    def train_one_epoch(self, seq_train_dataloader: DataLoader, verbose: bool) -> tuple:
        """Train the model for one epoch.

        Args:
            train_dataloader (DataLoader): DataLoader for the training set.
            verbose (bool): If True, shows progress using tqdm.

        Returns:
            tuple[float, float]: A tuple containing:
                - Train loss (float).
                - Spearman Sharpe for the training set (float).
        """
        self.model.train()
        total_loss = 0.0
        total_icir_loss = 0.0
        total_mse_loss = 0.0
        total_ranking_loss = 0.0

        y_total, preds_total = [], []

        for seq_batch in seq_train_dataloader:
            seq_x_batch = seq_batch["continuous"]
            seq_x_batch = {key: value.to(self.device) for key, value in seq_x_batch.items()}

            true_y = seq_batch["current"].to(self.device)

            self.optimizer.zero_grad()
            # self.lag_optimizer.zero_grad()
            # self.meta_optimizer.zero_grad()
            with torch.autocast(device_type="cuda"):
                pred_y = self.model(seq_x_batch)
                final_preds = pred_y["final_prediction"]
                loss, icir_loss, mse_loss, ranking_loss, _ = self.criterion(final_preds, true_y).values()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)

            self.optimizer.step()
            # self.lag_optimizer.step()
            # self.meta_optimizer.step()

            total_loss += loss.item()
            total_icir_loss += icir_loss.item()
            total_mse_loss += mse_loss.item()
            total_ranking_loss += ranking_loss.item()

            y_total.append(true_y)
            preds_total.append((final_preds).detach())

            torch.cuda.empty_cache()

        y_total = torch.cat(y_total).cpu().numpy().astype(np.float64)
        preds_total = torch.cat(preds_total).cpu().numpy().astype(np.float64)

        train_sharpe = rank_correlation_sharpe(y_total, preds_total)
        train_loss = total_loss / len(seq_train_dataloader)
        train_icir_loss = total_icir_loss / len(seq_train_dataloader)
        train_mse_loss = total_mse_loss / len(seq_train_dataloader)
        train_ranking_loss = total_ranking_loss / len(seq_train_dataloader)

        return train_loss, train_sharpe, train_icir_loss, train_mse_loss, train_ranking_loss

    @timer
    def validate_one_epoch(self, seq_val_dataloader: DataLoader, retrain_val_dataloader: DataLoader, verbose=False) -> tuple:
        """Validate the model on the validation set.

        Args:
            val_dataloader (DataLoader): DataLoader for the validation set.
            verbose (bool, optional): If True, shows progress using tqdm. Defaults to False.

        Returns:
            tuple[float, float]: A tuple containing:
                - Validation loss (float).
                - Spearman Sharpe for the validation set (float).
        """
        model = copy.deepcopy(self.model).to("cpu")

        losses, icir_losses, mse_losses, ranking_losses, all_y, all_preds = [], [], [], [], [], []

        for seq_batch, retrain_batch in zip(seq_val_dataloader, retrain_val_dataloader):
            # seq_batch = {key: value.to(self.device) for key, value in seq_batch.items()}
            # cs_batch = {key: value.to(self.device) for key, value in cs_batch.items()}
            seq_x_batch = seq_batch["continuous"]
            true_y = seq_batch["current"]

            # Predict
            model.eval()
            with torch.inference_mode():
                pred_y = model(seq_x_batch)
                final_preds = pred_y["final_prediction"]
                final_preds = torch.nan_to_num(final_preds)

                loss, icir_loss, mse_loss, ranking_loss, _ = self.criterion(final_preds, true_y).values()
                losses.append(loss.cpu().numpy())
                icir_losses.append(icir_loss.cpu().numpy())
                mse_losses.append(mse_loss.cpu().numpy())
                ranking_losses.append(ranking_loss.cpu().numpy())

                all_y.append(true_y)
                all_preds.append(final_preds)

            # Update weights
            if self.refit:
                retrain_seq_x_batch = retrain_batch["continuous"]
                retrain_true_y = retrain_batch["current"]

                self.refit_optimizer.zero_grad()
                # self.refit_lag_optimizer.zero_grad()
                # self.refit_meta_optimizer.zero_grad()

                model.train()
                pred_y = model(retrain_seq_x_batch)
                final_preds = pred_y["final_prediction"]

                loss = self.criterion(final_preds, retrain_true_y)["total_loss"]
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

                self.refit_optimizer.step()
                # self.refit_lag_optimizer.step()
                # self.refit_meta_optimizer.step()

        all_y = torch.cat(all_y).numpy().astype(np.float64)
        all_preds = torch.cat(all_preds).numpy().astype(np.float64)
        loss = np.mean(losses)
        val_icir_loss = np.mean(icir_losses)
        val_mse_loss = np.mean(mse_losses)
        val_ranking_loss = np.mean(ranking_losses)

        sharpe = rank_correlation_sharpe(all_y, all_preds)

        return loss, sharpe, val_icir_loss, val_mse_loss, val_ranking_loss

    def update(self, seq_X: Dict[int, np.array], true_y: np.array):
        """Update the model with new data.

        Args:
            X (np.array): Input data.
            y (np.array): Target variable.
            n_times (int): Number of time steps.
        """
        torch.manual_seed(self.random_seed)
        if self.lr_refit == 0.0:
            return

        seq_continuous_data = {
            lag: torch.tensor(np.nan_to_num(X, nan=0.0), dtype=torch.float32, device=self.device).unsqueeze(0) for lag, X in seq_X.items()
        }

        true_y = torch.tensor(np.nan_to_num(true_y, nan=0.0), dtype=torch.float32, device=self.device)
        self.model.train()

        self.refit_optimizer.zero_grad()
        # self.refit_lag_optimizer.zero_grad()
        # self.refit_meta_optimizer.zero_grad()
        with torch.autocast(device_type="cuda"):
            pred_y = self.model(seq_continuous_data)
            loss = self.criterion(pred_y["final_prediction"], true_y)["total_loss"]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
        self.refit_optimizer.step()
        # self.refit_lag_optimizer.step()
        # self.refit_meta_optimizer.step()

    def predict(self, seq_X: Dict[int, np.array]) -> tuple[np.array, torch.Tensor | list]:
        """Predict the target variable for the given input data.

        Args:
            X (np.array): Input data.

        Returns:
            tuple[np.array, torch.Tensor or list]: A tuple containing:
                - Predictions (np.array).
                - Hidden state (torch.Tensor or list).
        """
        torch.manual_seed(self.random_seed)
        seq_continuous_data = {
            lag: torch.tensor(np.nan_to_num(X, nan=0.0), dtype=torch.float32, device=self.device).unsqueeze(0) for lag, X in seq_X.items()
        }

        self.model.eval()
        with torch.inference_mode():
            preds = self.model(seq_continuous_data)
            preds = torch.nan_to_num(preds["final_prediction"])

        return preds.cpu().numpy().astype(np.float64)

In [10]:
# --- Prepare DataLoader ---
# Create the dataset

train_x = pl.scan_csv(CONFIG.TRAIN_X_PATH)
train_x = PREPROCESSOR(df=train_x)
train_x = train_x.clean()

features = FEATURE_ENGINEERING(df=train_x)
train_x: pl.DataFrame = features.create_market_features()

train_y = pl.scan_csv(CONFIG.TRAIN_Y_PATH)

curr_y = (
    train_y.with_columns([pl.col(CONFIG.LAGS[f"lag{i}"]).exclude(CONFIG.DATE_COL).shift(i + 1) for i in range(1, 5)])
    .with_columns(pl.all().exclude(CONFIG.DATE_COL).shift())
    .filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series())))
    .collect()
    .fill_null(0)
    .lazy()
)

y_feat = FEATURE_ENGINEERING(df=curr_y)
lags = y_feat._compute_lag_returns(df=curr_y)
market = y_feat._compute_market_stats(df=curr_y)
skew = y_feat._compute_return_skew(df=curr_y)

train_x = (
    train_x.join(curr_y.collect(), on=CONFIG.DATE_COL)
    .join(lags.collect(), on=CONFIG.DATE_COL)
    .join(market.collect(), on=CONFIG.DATE_COL)
    .join(skew.collect(), on=CONFIG.DATE_COL)
)


train_y = train_y.filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series()))).collect()
train_x = (
    train_x.with_columns([pl.when(pl.col(col).is_infinite()).then(0.0).otherwise(pl.col(col)).alias(col) for col in train_x.columns])
    .with_columns(pl.all().shrink_dtype())
    .filter(pl.col(CONFIG.DATE_COL).is_in(train_y.select(CONFIG.DATE_COL).to_series()))
    .with_columns(pl.col(CONFIG.DATE_COL).cast(pl.Int64))
    # .select([CONFIG.DATE_COL] + CONFIG.IMPT_COL)
)

retrain_x = train_x.with_columns(pl.all().exclude(CONFIG.DATE_COL).shift(5))
retrain_y = train_y.filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series()))).with_columns(
    pl.all().exclude(CONFIG.DATE_COL).shift(5)
)

train_y_arr = train_y.drop(CONFIG.DATE_COL).to_numpy()

train_y = pl.DataFrame(
    (train_y_arr - np.nanmean(train_y_arr, axis=1).reshape(train_y_arr.shape[0], -1))
    / np.nanstd(train_y_arr, axis=1).reshape(train_y_arr.shape[0], -1),
    schema=train_y.drop(CONFIG.DATE_COL).columns,
).insert_column(0, train_y.select(CONFIG.DATE_COL).to_series())

retrain_y_arr = retrain_y.drop(CONFIG.DATE_COL).to_numpy()
retrain_y = pl.DataFrame(
    (retrain_y_arr - np.nanmean(retrain_y_arr, axis=1).reshape(retrain_y_arr.shape[0], -1))
    / np.nanstd(retrain_y_arr, axis=1).reshape(retrain_y_arr.shape[0], -1),
    schema=train_y.drop(CONFIG.DATE_COL).columns,
).insert_column(0, train_y.select(CONFIG.DATE_COL).to_series())


_compute_lag_returns took 0.0125 seconds
_compute_autocorr_torch took 0.3736 seconds
_compute_obv took 0.0095 seconds
_compute_return_skew took 0.0025 seconds
_compute_volume_z took 0.0416 seconds
_compute_market_stats took 0.1341 seconds
_compute_atr took 0.0075 seconds
_compute_rolling took 0.0133 seconds
create_market_features took 2.1133 seconds
_compute_lag_returns took 0.0084 seconds
_compute_market_stats took 0.0115 seconds
_compute_return_skew took 0.0030 seconds


C:\Users\Admin\AppData\Local\Temp\ipykernel_47708\3323296322.py:59: RuntimeWarning: Mean of empty slice
  (retrain_y_arr - np.nanmean(retrain_y_arr, axis=1).reshape(retrain_y_arr.shape[0], -1))
c:\Users\Admin\anaconda3\envs\edmund\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [11]:
NN_model = NN(
    model=HierarchicalModel(input_dim={lag: len(features) for lag, features in CONFIG.LAG_FEATURES.items()}, lag_target_sizes=CONFIG.LAGS_TARGET),
    batch_size=CONFIG.BATCH_SIZE,
    lr=0.0005,
    lr_refit=0.0005,
    epochs=200,
    early_stopping_patience=10,
)

In [ ]:
# test_size = (
#     TEST_SIZE
#     if len(dates_unique) > TEST_SIZE * (n_splits + 1)
#     else len(dates_unique) // (n_splits + 1)
# )  # For testing purposes on small samples

dates_unique = train_x.filter(pl.col(CONFIG.DATE_COL) <= CONFIG.MAX_TRAIN_DATE).select(pl.col(CONFIG.DATE_COL).unique().sort()).to_series().to_numpy()
real_dates_unique = (
    train_x.filter(pl.col(CONFIG.DATE_COL) > CONFIG.MAX_TRAIN_DATE).select(pl.col(CONFIG.DATE_COL).unique().sort()).to_series().to_numpy()
)

cv = TimeSeriesSplit(n_splits=CONFIG.N_FOLDS)
cv_split = cv.split(dates_unique)

scores = []
models = []
for fold, (train_idx, valid_idx) in enumerate(cv_split):
    if fold <= 3:
        continue
    if CONFIG.VERBOSE:
        print("-" * 20 + f"Fold {fold}" + "-" * 20)
        print(f"Train dates from {dates_unique[train_idx].min()} to {dates_unique[train_idx].max()}")
        print(f"Valid dates from {dates_unique[valid_idx].min()} to {dates_unique[valid_idx].max()}")

    dates_train = dates_unique[train_idx]
    dates_valid = dates_unique[valid_idx]

    df_train = train_x.filter(pl.col(CONFIG.DATE_COL).is_in(dates_train))
    true_y = train_y.filter(pl.col(CONFIG.DATE_COL).is_in(dates_train))

    valid_period = range(min(dates_valid) - max(CONFIG.LAG_SEQ_LEN.values()) + 1, max(dates_valid) + 1)
    df_valid = train_x.filter(pl.col(CONFIG.DATE_COL).is_in(valid_period))
    df_valid_current_y = train_y.filter(pl.col(CONFIG.DATE_COL).is_in(valid_period))

    df_valid_retrain = retrain_x.filter(pl.col(CONFIG.DATE_COL).is_in(valid_period))
    df_valid_current_y_retrain = retrain_y.filter(pl.col(CONFIG.DATE_COL).is_in(valid_period))

    model_fold = copy.deepcopy(NN_model)

    model_fold.fit(
        train_set=(df_train, true_y),
        val_set=(df_valid, df_valid_current_y),
        retrain_set=(df_valid_retrain, df_valid_current_y_retrain),
        verbose=CONFIG.VERBOSE,
    )

    models.append(model_fold)

    torch.save(
        model_fold.model.state_dict(),
        f"C:/Users/Admin/Desktop/Personal-Projects/Kaggle/MITSUI&CO. Commodity Prediction Challenge/ensemble_{fold}.pth",
    )

    preds = []
    cnt_dates = 0
    model_save = copy.deepcopy(model_fold)

    model_fold.model.load_state_dict(
        torch.load(
            f"C:/Users/Admin/Desktop/Personal-Projects/Kaggle/MITSUI&CO. Commodity Prediction Challenge/ensemble_{fold}.pth",
            map_location=torch.device("cuda"),
        )
    )

    for date_id in tqdm(dates_valid):
        period = range(date_id - max(CONFIG.LAG_SEQ_LEN.values()) + 1, date_id + 1)

        df_valid_date = train_x.filter(pl.col(CONFIG.DATE_COL).is_in(period)).drop(CONFIG.DATE_COL)
        valid_lags = {lag: df_valid_date.select(features).to_numpy().astype(np.float64) for lag, features in CONFIG.LAG_FEATURES.items()}
        valid_lags = {lag: valid_lags[lag][-seq_len:] for lag, seq_len in CONFIG.LAG_SEQ_LEN.items()}

        if model_fold.refit and (cnt_dates > 0):
            df_upd = retrain_x.filter(pl.col(CONFIG.DATE_COL).is_in(period)).drop(CONFIG.DATE_COL)
            df_upd_lags = {lag: df_upd.select(features).to_numpy().astype(np.float64) for lag, features in CONFIG.LAG_FEATURES.items()}
            df_upd_lags = {lag: df_upd_lags[lag][-seq_len:] for lag, seq_len in CONFIG.LAG_SEQ_LEN.items()}

            df_upd_current_y = retrain_y.filter(pl.col(CONFIG.DATE_COL).is_in(date_id)).drop(CONFIG.DATE_COL).to_numpy()

            if len(df_upd) > 0:
                model_save.update(df_upd_lags, df_upd_current_y)

        preds_i = model_save.predict(valid_lags)

        preds += list(preds_i[-1].reshape(-1, CONFIG.NUM_TARGET_COLUMNS))

        cnt_dates += 1

    preds = np.array(preds)

    score = rank_correlation_sharpe(
        df_valid_current_y.drop(CONFIG.DATE_COL).to_numpy().astype(np.float64),
        preds,
    )
    scores.append(score)

    print(f"LAST VALIDIDATION Sharpe: {score:.5f}")

    model_real = copy.deepcopy(model_save)
    preds = []
    cnt_dates = 0
    for date_id in tqdm(real_dates_unique):
        # print(date_id)
        period = range(date_id - max(CONFIG.LAG_SEQ_LEN.values()) + 1, date_id + 1)
        df_valid_date = train_x.filter(pl.col(CONFIG.DATE_COL).is_in(period)).drop(CONFIG.DATE_COL)

        valid_lags = {lag: df_valid_date.select(features).to_numpy().astype(np.float64) for lag, features in CONFIG.LAG_FEATURES.items()}
        valid_lags = {lag: valid_lags[lag][-seq_len:] for lag, seq_len in CONFIG.LAG_SEQ_LEN.items()}

        if model_fold.refit and (cnt_dates > 0):
            df_upd = retrain_x.filter(pl.col(CONFIG.DATE_COL).is_in(period)).drop(CONFIG.DATE_COL)
            df_upd_lags = {lag: df_upd.select(features).to_numpy().astype(np.float64) for lag, features in CONFIG.LAG_FEATURES.items()}
            df_upd_lags = {lag: df_upd_lags[lag][-seq_len:] for lag, seq_len in CONFIG.LAG_SEQ_LEN.items()}

            df_upd_current_y = retrain_y.filter(pl.col(CONFIG.DATE_COL).is_in(date_id)).drop(CONFIG.DATE_COL).to_numpy()

            if len(df_upd) > 0:
                model_real.update(df_upd_lags, df_upd_current_y)

        # print(df_upd[:, 0])
        # print(df_upd_current_y[:, -1])
        # print(df_upd_true_delta[:, -1])
        preds_i = model_real.predict(valid_lags)

        preds += list(preds_i[-1].reshape(-1, CONFIG.NUM_TARGET_COLUMNS))
        # print(preds_i[-1].reshape(-1, CONFIG.NUM_TARGET_COLUMNS))

        cnt_dates += 1
        # if cnt_dates == 2:
        #     break

    preds = np.array(preds)

    score = rank_correlation_sharpe(
        train_y.filter(pl.col(CONFIG.DATE_COL).is_in(real_dates_unique)).drop(CONFIG.DATE_COL).to_numpy().astype(np.float64),
        preds,
    )
    scores.append(score)
    print(f"REAL Sharpe: {score:.5f}")

--------------------Fold 4--------------------
Train dates from 2 to 1522
Valid dates from 1523 to 1826
Device: cuda:0
Epoch | Train Loss | Train ICIR Loss | Train MSE Loss | Train Ranking Loss | Val Loss | Val ICIR Loss | Val MSE Loss | Val Ranking Loss | Train sharpe | Val sharpe |   LR   
------------------------------------------------------------


In [ ]:
retrain_ds = SequentialDataset(df_valid_retrain, df_valid_current_y_retrain)
pl.DataFrame(next(iter(retrain_ds))[0][4].numpy())

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_63,column_64,column_65,column_66,column_67,column_68,column_69,column_70,column_71,column_72,column_73,column_74,column_75,column_76,column_77,column_78,column_79,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89,column_90,column_91,column_92,column_93,column_94,column_95,column_96,column_97,column_98,column_99
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
-0.600796,-0.914647,0.030829,-0.708012,-0.054668,-0.948001,-1.359432,-0.146292,-0.137402,-0.850851,0.078772,-0.123519,0.000983,-0.884438,1.125735,1.143567,1.159554,-0.505677,1.106106,-0.168141,-1.12163,-0.141626,-0.418873,-2.22735,-0.018628,0.876863,-0.263154,-0.659306,-0.949431,-1.160557,-0.547408,-0.144907,-0.32823,0.011029,0.008649,-1.469505,-0.192384,…,0.416911,0.49945,0.415239,0.01134,0.505069,0.011571,0.011571,-0.065728,0.428339,0.427753,-1.43725,-1.056561,-0.017627,0.110134,0.15424,0.503649,0.620564,0.06471,0.399062,0.604878,0.691247,-0.02008,0.693985,0.638001,-1.114075,0.182135,0.662117,0.721626,-0.987616,0.682037,-0.065506,1.024815,-0.410165,1.093586,-0.052384,0.030002,0.118704
-0.331305,-0.799951,0.04221,-0.609153,0.222382,-1.083803,-1.329361,-0.524006,-0.866288,-0.78775,-0.705297,0.011299,-0.016925,-0.681306,1.074381,1.087834,1.111351,-0.04803,1.053658,-0.819396,-0.831125,-0.809522,-0.274026,-2.369183,-0.068554,0.866089,-1.459333,-0.513191,-0.818227,-1.076879,-0.287044,-1.180784,-0.213305,-0.002656,-0.586793,-1.471487,-1.138588,…,0.343683,0.436692,0.33969,-0.005089,0.441262,-0.024865,-0.024865,-0.018148,0.358543,0.355994,-1.439224,-0.91452,0.011029,-0.09682,-1.293827,0.296467,0.540513,-0.136164,-0.716517,0.52334,0.608413,0.168394,0.620309,0.637208,-1.226812,0.063022,0.663466,0.724069,-0.763981,0.68042,-0.068423,0.949595,-1.30037,1.074553,-0.389326,0.056125,-0.120663
-0.672513,-0.636656,0.046717,-0.600888,0.41067,-0.007206,-1.275968,-0.859563,-1.660442,-0.744369,-0.602834,0.000385,0.01921,-0.459605,1.024923,1.03614,1.055555,0.60446,1.007326,-1.579592,-0.377803,-1.605468,-0.090251,2.471409,0.191179,0.707784,-1.358112,-0.466072,-0.894953,-0.814633,-0.734001,-0.546881,-0.124805,0.021448,-0.777764,-1.104306,-0.532989,…,0.26818,0.357569,0.263673,-0.026372,0.358072,-0.02419,-0.02419,-0.026771,0.275264,0.273948,-1.451764,-0.862717,-0.002656,-0.346436,0.475959,0.182605,0.4661,-0.360702,0.214843,0.454266,0.53341,0.190971,0.535629,0.640734,-1.364923,0.327149,0.667453,0.727474,-0.545925,0.684033,-0.044396,0.847165,-1.70006,1.060013,-0.580176,0.024639,-0.258854
-0.669441,-0.958308,0.041761,-0.694853,0.48666,-0.399988,-0.731586,-0.852627,-2.475681,-0.705136,-0.313858,0.07329,-0.023334,-0.469034,0.978591,0.98603,1.011364,0.021094,0.956699,-2.387139,-0.162747,-2.413514,0.981804,2.242544,0.101565,0.63733,-1.347286,-0.552309,-0.894854,-0.839749,-1.0248,-1.514108,-0.186853,0.007691,-0.70138,-0.926617,-1.330135,…,0.214499,0.285683,0.209884,-0.039141,0.285819,0.022193,0.022193,0.014136,0.230739,0.231165,-1.503957,-1.070332,0.021448,-0.551869,0.168374,0.124773,0.392771,-0.570181,-0.43412,0.375534,0.457342,0.102894,0.465667,0.642076,-1.394,0.275905,0.663425,0.725267,-0.496921,0.686843,-0.002049,0.731776,-1.717889,1.030099,-0.786002,-0.005382,-0.331825
0.514794,-1.189119,0.022192,-0.588277,-0.064362,0.077836,-0.398386,-0.778215,-1.876163,-0.639785,0.000009,0.172795,0.027207,-0.403682,0.921649,0.928434,0.955571,-0.079817,0.901019,

In [ ]:
train_x.filter(pl.col(CONFIG.DATE_COL).is_in(range(date_id - max(CONFIG.LAG_SEQ_LEN.values()) + 1, date_id + 1))).select(
    [CONFIG.DATE_COL] + CONFIG.LAG_FEATURES[1]
)

date_id,LME_CA_Close_vol_20,US_Stock_SPTL_adj_close_log_ret_return_lag_3,target_334,target_334_return_lag_0,FX_AUDUSD_log_ret_return_lag_3,US_Stock_VGLT_adj_close_log_ret_return_lag_3,FX_USDCHF_log_ret_return_lag_4,US_Stock_CCJ_adj_high_log_ret,US_Stock_CCJ_adj_high_log_ret_return_lag_0,US_Stock_VGK_adj_high_log_ret_return_lag_4,US_Stock_URA_adj_high_log_ret_return_lag_1,US_Stock_IGSB_adj_high_log_ret_return_lag_4,US_Stock_OKE_adj_close_vol_5,target_353,target_353_return_lag_0,US_Stock_VEA_adj_high_log_ret_return_lag_4,FX_GBPAUD_log_ret_return_lag_2,target_352_return_lag_5,US_Stock_EWY_adj_high_log_ret_return_lag_4,US_Stock_VALE_adj_high_log_ret_return_lag_4,target_342_return_lag_4,US_Stock_EFA_adj_high_log_ret_return_lag_4,US_Stock_GLD_adj_high_log_ret_return_lag_1,target_282_return_lag_4,US_Stock_EWJ_adj_high_log_ret_return_lag_4,US_Stock_RSP_adj_low_log_ret_return_lag_4,US_Stock_BNDX_adj_close_log_ret_return_lag_5,target_391_return_lag_5,US_Stock_EWY_adj_open_log_ret_return_lag_4,US_Stock_VALE_adj_open_log_ret_return_lag_4,US_Stock_BNDX_adj_low_vol_5,FX_USDJPY_log_ret_return_lag_3,US_Stock_IGSB_adj_high_log_ret_return_lag_2,US_Stock_SPTL_adj_low_log_ret_return_lag_3,LME_PB_Close_log_ret_return_lag_3,target_293_return_lag_3,…,US_Stock_YINN_adj_open_log_ret_return_lag_4,US_Stock_VCSH_adj_low_log_ret_return_lag_4,US_Stock_KMI_adj_low_log_ret_return_lag_1,US_Stock_BKR_adj_close_log_ret,US_Stock_BKR_adj_close_log_ret_return_lag_0,US_Stock_BP_adj_high_log_ret_return_lag_4,US_Stock_BKR_adj_close_log_ret_return_lag_4,US_Stock_CVX_adj_low_log_ret_return_lag_4,US_Stock_SPIB_adj_open_log_ret_return_lag_5,US_Stock_SHY_adj_low_log_ret_return_lag_3,US_Stock_YINN_adj_low_log_ret_return_lag_4,US_Stock_BSV_adj_low_log_ret_return_lag_3,US_Stock_CCJ_adj_close_log_ret_return_lag_4,target_355_return_lag_2,US_Stock_FXI_adj_low_log_ret_return_lag_4,US_Stock_BNDX_adj_low_log_ret_return_lag_3,JPX_Gold_Standard_Futures_Low_log_ret,JPX_Gold_Standard_Futures_Low_log_ret_return_lag_0,target_396_return_lag_5,US_Stock_EWJ_adj_low_log_ret_return_lag_4,JPX_Platinum_Standard_Futures_Close_log_ret_return_lag_4,JPX_Gold_Mini_Futures_Low_log_ret,JPX_Gold_Mini_Futures_Low_log_ret_return_lag_0,US_Stock_VGLT_adj_low_log_ret_return_lag_3,US_Stock_VWO_adj_close_log_ret,US_Stock_VWO_adj_close_log_ret_return_lag_0,FX_USDJPY_sma_5_ratio,US_Stock_MPC_adj_close_vol_20,JPX_Platinum_Standard_Futures_Low_log_ret,JPX_Platinum_Standard_Futures_Low_log_ret_return_lag_0,US_Stock_TECK_adj_open_log_ret,US_Stock_TECK_adj_open_log_ret_return_lag_0,US_Stock_LYB_adj_low_log_ret_return_lag_4,FX_AUDCAD_log_ret_return_lag_2,US_Stock_EWZ_adj_low_log_ret,US_Stock_EWZ_adj_low_log_ret_return_lag_0,US_Stock_FXI_adj_open_log_ret_return_lag_5
i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1492,-0.511044,0.173562,-0.033112,-0.033112,1.533998,0.234326,1.023351,-2.188488,-2.188488,-0.663893,-2.809797,-0.074308,1.073938,-0.022895,-0.022895,-0.519893,-0.154931,-0.023216,-0.240476,-0.428015,0.023171,-0.455075,-2.550884,-0.03502,0.215218,-0.611564,-0.224083,0.027589,-0.173789,-0.15977,-0.008468,-0.157877,0.954141,-1.089345,1.486385,-0.037555,…,0.161582,-1.27943,-2.127097,-0.125517,-0.125517,0.738528,0.674825,0.727557,0.234615,0.581767,-0.088414,-0.000459,-0.356188,0.032037,-0.118028,-1.095543,-2.022053,-2.022053,0.026556,0.024484,0.329002,-2.035179,-2.035179,-1.097548,-1.284433,-1.284433,-0.23474,-0.97524,-2.99021,-2.99021,-1.378905,-1.378905,0.077251,1.309706,-1.638625,-1.638625,-0.276408
1493,-0.312962,0.101284,-0.016007,-0.016007,0.02741,0.097307,-1.096954,-1.957091,-1.957091,0.558081,-1.654699,-0.45129,1.165789,-0.038005,-0.038005,0.381586,0.147775,0.013702,0.254698,0.281916,-0.008606,0.401456,-0.312567,0.010238,-0.371636,0.65835,-0.746884,0.027614,

In [ ]:
for date_id in tqdm(dates_valid):
    df_valid_date = train_x.filter(pl.col(CONFIG.DATE_COL).is_in(range(date_id - max(CONFIG.LAG_SEQ_LEN.values()) + 1, date_id + 1))).drop(
        CONFIG.DATE_COL
    )
    valid_lags = {lag: df_valid_date.select(features).to_numpy().astype(np.float64) for lag, features in CONFIG.LAG_FEATURES.items()}
    valid_lags = {lag: valid_lags[lag][-seq_len:] for lag, seq_len in CONFIG.LAG_SEQ_LEN.items()}
    break

  0%|          | 0/304 [00:00<?, ?it/s]

In [ ]:
# Step 1: Prepare the numpy input
X_np = (
    df_valid.drop(
        CONFIG.DATE_COL,
    )
    .to_numpy()
    .reshape(-1, 8, 173)
)

# Step 2: Create leaf tensor on CUDA with requires_grad=True
X_sample = torch.tensor(X_np, dtype=torch.float32, device="cuda", requires_grad=True)  # ✅ single step

# Step 3: Put model into train mode to support CuDNN RNN backward
model_real.model.train()

# Step 4: Forward and backward pass
output = model_real.model(X_sample)  # (B, ...)
loss = output.mean()
loss.backward()

# Step 5: Retrieve input gradients — now this will work without warnings
grads = X_sample.grad.detach().cpu().numpy()  # shape: (B, T, F)
importance = np.abs(grads).mean(axis=(0, 1))  # shape: (F,)
importance /= importance.sum()

ValueError: cannot reshape array of size 2831456 into shape (8,173)

In [ ]:
pl.DataFrame(importance).with_columns(pl.Series(name="feats", values=df_train.drop(CONFIG.DATE_COL).columns)).sort(by="column_0", descending=True)

column_0,feats
f32,str
0.011635,"""US_Stock_GLD_adj_low_log_ret_r…"
0.011528,"""US_Stock_NUGT_adj_close_log_re…"
0.010101,"""US_Stock_IGSB_adj_open_log_ret…"
0.00998,"""US_Stock_RY_adj_high_log_ret"""
0.009936,"""US_Stock_MPC_adj_low"""
…,…
0.003558,"""US_Stock_VCIT_adj_open_log_ret…"
0.003554,"""US_Stock_AMP_adj_low_log_ret"""
0.003419,"""US_Stock_LYB_adj_close_log_ret…"
